<a href="https://colab.research.google.com/github/dasmiq/cs6120-assignment2/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [21]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
import re
from scipy.sparse import lil_matrix

In [22]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [23]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  
  # collect takens
  all_tokens = set()
  tokenized_docs = []

  for doc in corpus:
        text = doc["text"].lower()
        tokens = re.findall(r"[a-zA-Z]+", text)
        tokenized_docs.append(tokens)
        all_tokens.update(tokens)

  # build vocabulary 
  vocab = sorted(all_tokens)
  token_index = {token: idx for idx, token in enumerate(vocab)}

  # create sparse matrix
  num_docs = len(corpus)
  num_vocab = len(vocab)
  texts = lil_matrix((num_docs, num_vocab), dtype=int)
  for doc_id, tokens in enumerate(tokenized_docs):
      for token in tokens:
          if token in token_index:
              texts[doc_id, token_index[token]] += 1

  # collect genres
  genres = [doc["genre"] for doc in corpus]
  
  return texts,genres,vocab

In [24]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
    """
    Create feature matrix with additional features for Shakespeare plays.
    Features included:
    1. Unigrams: basic bag-of-words representation.
    2. Bigrams: consecutive word pairs to capture local context.
    3. Suffix features: special suffix indicators (e.g., -eth, -est, -less, -tion).
    4. Emotion lexicon: counts of positive and negative words, "pos" corresponds to comedy,
       "neg" corresponds to tragedy.
    """

    # collect tokens
    tokenized_docs = []
    all_features = set()
    suffixes = ["eth", "est", "less", "tion", "ful", "ous", "y"]

    # simple positive/negative lexicons
    pos_words = ["love", "happy", "joy", "merry", "laugh"]
    neg_words = ["death", "blood", "sorrow", "revenge", "kill"]

    for doc in corpus:
        text = doc["text"].lower()
        tokens = re.findall(r"[a-zA-Z]+", text)
        features = []

        # Unigram features
        features.extend(tokens)

        # Bigram features
        for i in range(len(tokens) - 1):
            bigram = tokens[i] + "_" + tokens[i+1]
            features.append(bigram)

        # Suffix features
        for suf in suffixes:
            count = sum(1 for t in tokens if t.endswith(suf))
            if count > 0:
                features.append(f"<SUFFIX_{suf}>")

        # Emotion lexicon features
        pos_count = sum(tokens.count(w) for w in pos_words)
        neg_count = sum(tokens.count(w) for w in neg_words)
        if pos_count > 0:
            features.append("<EMO_POSITIVE>")
        if neg_count > 0:
            features.append("<EMO_NEGATIVE>")

        tokenized_docs.append(features)
        all_features.update(features)

    # build vocabulary 
    vocab = sorted(all_features)
    feature_index = {feature: idx for idx, feature in enumerate(vocab)}

    # create sparse matrix
    num_docs = len(corpus)
    num_features = len(vocab)
    texts = lil_matrix((num_docs, num_features), dtype=int)
    for doc_id, features in enumerate(tokenized_docs):
        for feature in features:
            if feature in feature_index:
                texts[doc_id, feature_index[feature]] += 1

    # collect genres
    genres = [doc["genre"] for doc in corpus]
    
    return texts,genres,vocab

In [25]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [26]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [27]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['you', 'helena', 'duke', 'prospero', 'i', 'sir', 'leontes', 'a', 'of', 'pede', 'pressing', 'presses', 'pressed', 'press', 'president', 'preserving', 'pressure', 'preservers', 'preserved', 'preserve']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'o', 'brutus', 'lear', 'ham', 'and', 'rom', 'what', 'the', 'preordinance', 'pressing', 'presses', 'pressed', 'press', 'president', 'preserving', 'preservers']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', 'ham', 'our', 'desdemona', 'r

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [28]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.692308
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'duke', 'you', 'for', 'your', 'i', 'a', 'leontes', 'me', 'of', 'queen_over', 'queen_pisanio', 'queen_on', 'queen_of', 'queen_paris', 'queen_part', 'queen_paulina', 'queen_one']
The most informative terms for neg are: ['our', 'him', 'iago', 'thy', 'rom', 's', 'imogen', 'what', 'brutus', 'his', 'ham', 'lear', 'o', 'and', 'quarrelsome_and', 'queen_not', 'queen_nothing', 'queen_now', 'queen_o', 'queen_of']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'a', 'prospero', 'helena', 'your', 'sir', 'antonio', 'for', 'hermia', 'leontes', 'lysander', 'ariel', 'sebastian', 'stephano', 'demetrius', 'camillo', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'brutus', 'imogen', 'what', 'our', 'his', 'lear', 'cassio', 'o', 'desdemona', 'ham

**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?

1. The basic model achieves 61.5% (L1) and 76.9% (L2) accuracy, The fancy model improves the L1 result to 69.2%, while the L2 accuracy remains the same (76.9%). It shows richer features (bigrams, suffixes, emotion words) mainly help under sparsity constraints (L1), while L2 benefits less.
2. For comedies (pos): pronouns (you, your) are strong indicators. Bigrams like queen_over also appear, capturing contextual patterns, while for tragedy (neg)pronouns (him, our) are strong indicators.
3. The most informative features learned by the classifier are character names (e.g., Iago, Othello, Helena, Prospero) and pronouns (I, you, him, thy, our), which strongly differentiate comedies and tragedies. Interestingly, suffix-based features and emotion lexicon words rarely appear among the top indicators. This suggests that Shakespeare’s plays express genre differences more through characters and pronoun usage rather than explicit emotional words or suffix patterns.